In [44]:
import pandas as pd
import numpy as np
from datetime import timedelta
import datetime as dt
np.seterr(divide='ignore', invalid='ignore')
import plotly.express as px
import plotly.graph_objects as go

In [45]:
#refrence to csv file 
csv_path = "desktop/Pandemic (H1N1) 2009.csv"

#CSV into pandas frame
h1n1_df = pd.read_csv(csv_path, parse_dates=["Update Time"], encoding = 'unicode_escape')

In [46]:
h1n1_df

,Country,Cases,Deaths,Update Time
0,Algeria,5,0.0,2009-07-06 09:00:00
1,Antigua and Barbuda,2,0.0,2009-07-06 09:00:00
2,Argentina,2485,60.0,2009-07-06 09:00:00
3,Australia,5298,10.0,2009-07-06 09:00:00
4,Austria,19,0.0,2009-07-06 09:00:00
...,...,...,...,...
1817,Thailand,2,0.0,2009-05-23 08:00:00
1818,Turkey,2,0.0,2009-05-23 08:00:00
1819,United Kingdom,117,0.0,2009-05-23 08:00:00
1820,United States of America,6552,9.0,2009-05-23 08:00:00


In [58]:
# number of deaths by country 
h1n1_df = h1n1_df.sort_values(by="Deaths", ascending=False).reset_index(drop=True)
h1n1_df.head()

,Country,Cases,Deaths,Update Time
0,Grand Total,94512,429.0,2009-07-06 09:00:00
1,Grand Total,89921,382.0,2009-07-03 09:00:00
2,Grand Total,77201,332.0,2009-07-01 09:00:00
3,Grand Total,59814,263.0,2009-06-26 07:00:00
4,Grand Total,55867,238.0,2009-06-24 07:00:00


In [60]:
#cases by Country
h1n1_df = h1n1_df.sort_values(by="Cases", ascending=False).reset_index(drop=True)
h1n1_df.head()

,Country,Cases,Deaths,Update Time
0,Grand Total,94512,429.0,2009-07-06 09:00:00
1,Grand Total,89921,382.0,2009-07-03 09:00:00
2,Grand Total,77201,332.0,2009-07-01 09:00:00
3,Grand Total,59814,263.0,2009-06-26 07:00:00
4,Grand Total,55867,238.0,2009-06-24 07:00:00


In [24]:
h1n1_df = h1n1_df.loc[:,['Country','Cases','Deaths','Update Time']]
h1n1_df['Country'].nunique()

152

In [31]:
h1n1_df.set_index("Country")
h1n1_df

,Country,Cases,Deaths,Update Time
0,Algeria,5,0.0,2009-07-06 09:00:00
1,Antigua and Barbuda,2,0.0,2009-07-06 09:00:00
2,Argentina,2485,60.0,2009-07-06 09:00:00
3,Australia,5298,10.0,2009-07-06 09:00:00
4,Austria,19,0.0,2009-07-06 09:00:00
...,...,...,...,...
1817,Thailand,2,0.0,2009-05-23 08:00:00
1818,Turkey,2,0.0,2009-05-23 08:00:00
1819,United Kingdom,117,0.0,2009-05-23 08:00:00
1820,United States of America,6552,9.0,2009-05-23 08:00:00


In [32]:
h1n1_df.set_index("Country", inplace = True)
h1n1_df

,Cases,Deaths,Update Time
Country,,,
Algeria,5,0.0,2009-07-06 09:00:00
Antigua and Barbuda,2,0.0,2009-07-06 09:00:00
Argentina,2485,60.0,2009-07-06 09:00:00
Australia,5298,10.0,2009-07-06 09:00:00
Austria,19,0.0,2009-07-06 09:00:00
...,...,...,...
Thailand,2,0.0,2009-05-23 08:00:00
Turkey,2,0.0,2009-05-23 08:00:00
United Kingdom,117,0.0,2009-05-23 08:00:00


In [40]:
#Create Dataset that lists all countries
h1n1_df = h1n1_df.loc[:,['Country','Cases','Deaths','Update Time']]


#Find start date of infections in YYY-DD-MM format
start_day = ('Country').agg('min').reset_index(drop=False)
start_day = start_day.loc[:,['Country', 'Update Time']]

#Find todays date in YYYY-DD-MM format
today = dt.datetime.today()

#Days since first infection date
days_difference = []
days_difference = (today - start_day['Update Time']).dt.days
days_difference

#Merge on index
start_day = start_day.merge(days_difference, left_index=True, right_index=True)
start_day = start_day.rename(columns={'Country': 'Country', 
                                      'Date_x': 'Start Date', 
                                      'Date_y': 'Days Since 1st Case'})

start_day['Weeks Since 1st Case'] = round(start_day['Days Since 1st Case'] / 7, 2)

start_day.sort_values(by='Days Since 1st Case', ascending=False)\
.reset_index(drop=True).style.background_gradient(cmap='Reds')
#start_day

AttributeError: 'str' object has no attribute 'agg'

In [ ]:
#store Country, days since 1st casr and weeks since 1st case

start_day = start_day.loc[:, ['Country', 'Days Since 1st Case', 'Weeks Since 1st Case']]
start_day

In [63]:
#Country deep Dive Italy 

italy = h1n1_df.loc[h1n1_df["Country"] == "Italy", :]
italy = italy.reset_index(drop=True)

#Find start date of infections 
italy_start = italy["Update Time"].reset_index(drop=True)[0]

#Find todays date in YYYY-DD-MM format
today = dt.datetime.today()

#days since first infection date
difference = today - italy_start
difference = difference.days
difference


#Find Daily increase in Cases and Deaths
it_confirmed_pct_change = round(italy['Cases'].pct_change() * 100, 2)
it_death_change = round(italy['Deaths'].pct_change() * 100, 2)

#Find Daily Percentage Increase in Confirmed cases and Deaths
it_confirmed_perday = italy['Cases'].diff()
it_death_perday = italy['Deaths'].diff()

italy_percent_change = pd.DataFrame

italy_percent = pd.DataFrame({
                   "New Confirmed Cases Per Day": it_confirmed_perday,
                   "Confirmed Percent Change" : it_confirmed_pct_change,
                   "New Confirmed Deaths Per Day": it_death_perday,
                   "Death Percent Change" : it_death_change,
                    })
    
italy_percent.fillna(value=0)

,New Confirmed Cases Per Day,Confirmed Percent Change,New Confirmed Deaths Per Day,Death Percent Change
0,0.0,0.00,0.0,0.0
1,-16.0,-10.96,0.0,0.0
2,-7.0,-5.38,0.0,0.0
3,-11.0,-8.94,0.0,0.0
4,-10.0,-8.93,0.0,0.0
5,-6.0,-5.88,0.0,0.0
6,-8.0,-8.33,0.0,0.0
7,0.0,0.00,0.0,0.0
8,-16.0,-18.18,0.0,0.0
9,-5.0,-6.94,0.0,0.0


In [66]:

#Merge DataSets together into 1 df
italy_df = italy.merge(italy_percent, left_index=True, right_index=True)
italy_df = italy_df.fillna(value=0)

In [68]:
italy_df

,Country,Cases,Deaths,Update Time,New Confirmed Cases Per Day,Confirmed Percent Change,New Confirmed Deaths Per Day,Death Percent Change
0,Italy,146,0.0,2009-07-06 09:00:00,0.0,0.00,0.0,0.0
1,Italy,130,0.0,2009-07-03 09:00:00,-16.0,-10.96,0.0,0.0
2,Italy,123,0.0,2009-07-01 09:00:00,-7.0,-5.38,0.0,0.0
3,Italy,112,0.0,2009-06-29 09:00:00,-11.0,-8.94,0.0,0.0
4,Italy,102,0.0,2009-06-26 07:00:00,-10.0,-8.93,0.0,0.0
5,Italy,96,0.0,2009-06-24 07:00:00,-6.0,-5.88,0.0,0.0
6,Italy,88,0.0,2009-06-22 07:00:00,-8.0,-8.33,0.0,0.0
7,Italy,88,0.0,2009-06-19 07:00:00,0.0,0.00,0.0,0.0
8,Italy,72,0.0,2009-06-17 12:00:00,-16.0,-18.18,0.0,0.0
9,Italy,67,0.0,2009-06-15 17:00:00,-5.0,-6.94,0.0,0.0


In [74]:
#Country deep United States of America

UnitedStatesofAmerica = h1n1_df.loc[h1n1_df["Country"] == "United States of America", :]
UnitedStatesofAmerica = UnitedStatesofAmerica.reset_index(drop=True)

#Find start date of infections
UnitedStatesofAmerica_start = UnitedStatesofAmerica["Update Time"].reset_index(drop=True)[0]


#Find todays date in YYYY-DD-MM format
today = dt.datetime.today()

#days since first infection date
difference = today - UnitedStatesofAmerica_start
difference = difference.days
difference


#Find Daily increase in Cases and Deaths
it_confirmed_pct_change = round(UnitedStatesofAmerica['Cases'].pct_change() * 100, 2)
it_death_change = round(UnitedStatesofAmerica['Deaths'].pct_change() * 100, 2)

#Find Daily Percentage Increase in Confirmed cases and Deaths
it_confirmed_perday = UnitedStatesofAmerica['Cases'].diff()
it_death_perday = UnitedStatesofAmerica['Deaths'].diff()

UnitedStatesofAmerica_percent_change = pd.DataFrame

UnitedStatesofAmerica_percent = pd.DataFrame({
                   "New Confirmed Cases Per Day": it_confirmed_perday,
                   "Confirmed Percent Change" : it_confirmed_pct_change,
                   "New Confirmed Deaths Per Day": it_death_perday,
                   "Death Percent Change" : it_death_change,
})
    
UnitedStatesofAmerica_percent.fillna(value=0)

,New Confirmed Cases Per Day,Confirmed Percent Change,New Confirmed Deaths Per Day,Death Percent Change
0,0.0,0.00,0.0,0.00
1,0.0,0.00,0.0,0.00
2,-6185.0,-18.24,-43.0,-25.29
3,0.0,0.00,0.0,0.00
4,-6268.0,-22.61,-40.0,-31.50
5,0.0,0.00,0.0,0.00
6,0.0,0.00,0.0,0.00
7,-3594.0,-16.76,-43.0,-49.43
8,0.0,0.00,1.0,2.27
9,-4638.0,-25.98,-18.0,-40.00


In [76]:
us_df = UnitedStatesofAmerica.merge(UnitedStatesofAmerica_percent, left_index=True, right_index=True)
us_df = us_df.fillna(value=0)

In [77]:
us_df

,Country,Cases,Deaths,Update Time,New Confirmed Cases Per Day,Confirmed Percent Change,New Confirmed Deaths Per Day,Death Percent Change
0,United States of America,33902,170.0,2009-07-03 09:00:00,0.0,0.00,0.0,0.00
1,United States of America,33902,170.0,2009-07-06 09:00:00,0.0,0.00,0.0,0.00
2,United States of America,27717,127.0,2009-06-29 09:00:00,-6185.0,-18.24,-43.0,-25.29
3,United States of America,27717,127.0,2009-07-01 09:00:00,0.0,0.00,0.0,0.00
4,United States of America,21449,87.0,2009-06-26 07:00:00,-6268.0,-22.61,-40.0,-31.50
5,United States of America,21449,87.0,2009-06-22 07:00:00,0.0,0.00,0.0,0.00
6,United States of America,21449,87.0,2009-06-24 07:00:00,0.0,0.00,0.0,0.00
7,United States of America,17855,44.0,2009-06-19 07:00:00,-3594.0,-16.76,-43.0,-49.43
8,United States of America,17855,45.0,2009-06-15 17:00:00,0.0,0.00,1.0,2.27
9,United States of America,13217,27.0,2009-06-12 07:00:00,-4638.0,-25.98,-18.0,-40.00
